# Summary

Start tying openai and youtube functionality together to manage the punctuation process.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
from youtube_transcript_api import YouTubeTranscriptApi, NoTranscriptFound

from jabberwocky.config import C
from jabberwocky.openai_utils import query_gpt3
from jabberwocky.utils import load_prompt
from jabberwocky.youtube import realign_punctuated_text, get_transcripts, \
    text_segment, video_id
from htools import *

In [3]:
cd_root()

Current directory: /Users/hmamin/jabberwocky


In [195]:
class UnpunctuatedTranscript:
    
    def __init__(self, df_gen, **kwargs):
        self.df_gen = df_gen
        self.df_punct = self.df_gen.copy()
        self.df_punct['text'] = np.nan
        # TODO: adjust so prompt template saved separately from prompt_kwargs.
        # TODO: adjust load_prompt to have option to skip .format call 
        # if no text passed in.
        self.prompt_kwargs = load_prompt('punctuate')
        self.prompt_fmt = self.prompt_kwargs.pop('prompt')
        self.prompt_kwargs.update(kwargs)
        
    @property
    def df(self):
        # TODO: maybe should return self.df_punct, possibly after punct query? 
        # Unsure.
        return self.df_gen
    
    def _punctuate_chunk(self, df_chunk, **kwargs) -> str:
        text = ' '.join(df_chunk.text)
        prompt = self.prompt_fmt.format(text).rstrip()
        # n_tokens is roughly 1.33 * n_words. Add buffer for punctuation.
        prompt_kwargs = {**self.prompt_kwargs,
                         **kwargs, 
                         'max_tokens': int(len(text.split()) * 2)}
        _, text_punct = query_gpt3(prompt, **prompt_kwargs)
        return text_punct
    
#     def punctuated_chunk(self, start_idx, end_idx, align_kwargs=None, 
#                         **query_kwargs):
#         df_chunk = self.df_gen.loc[start_idx:end_idx]
#         for idx in self.na_index_chunks(self.df_punct.loc[start_idx:end_idx]):
#             print(idx)
#         return
        
        
#         text_punct = self._punctuate_chunk(df_chunk, **query_kwargs)
#         df_chunk_punct = realign_punctuated_text(df_chunk, text_punct, 
#                                                  **ifnone(align_kwargs, {}))
#         # Must use 'values' attribute because realignment func resets index.
#         self.df_punct.loc[start_idx:end_idx, 'text'] = df_chunk_punct.text.values
#         return self.df_punct.loc[start_idx:end_idx, :]
    
    # This version only punctuates rows of the relevant chunk that haven't 
    # been previously punctuated. While this is faster and cheaper, I've seen
    # some hints that the punctuation task may work better when we pass it
    # long-ish chunks of text and not little partial snippets. So it might
    # actually be better not to do this? I was going to say it might do better
    # if given full sentences rather than fragments, but I guess we can't 
    # easily extract those without doing the actual punctuation step.
    def punctuated_chunk(self, start_idx, end_idx, punctuate, 
                         align_kwargs=None, **query_kwargs):
        if not punctuate:
            return self.df_gen.loc[start_idx:end_idx, :]
        for idx in self.na_index_chunks(self.df_punct.loc[start_idx:end_idx]):
            df_chunk = self.df_gen.loc[idx]
            text_punct = self._punctuate_chunk(df_chunk, **query_kwargs)
            df_chunk_punct = realign_punctuated_text(
                df_chunk, text_punct, **ifnone(align_kwargs, {})
            )
            # Use 'values' attribute because realignment func resets index.
            self.df_punct.loc[idx, 'text'] = df_chunk_punct.text.values
        return self.df_punct.loc[start_idx:end_idx, :]
    
    @staticmethod
    def na_index_chunks(chunk) -> list:
        """Given a chunk of a df that may contain null text rows, return a 
        list of lists where each nested list contains the indices of a 
        contiguous chunk of null rows.
        """
        nans = chunk[chunk.text.isnull()]
        if nans.empty: return []
        last_idx = nans.index[-1]
        res = []
        curr_chunk = []
        prev = None
        for idx in nans.index:
            if prev is None or (idx == prev + 1): 
                curr_chunk.append(idx)
            else:
                res.append(curr_chunk)
                curr_chunk = [idx]
            if idx == last_idx:
                res.append(curr_chunk)
            prev = idx
        return res

In [196]:
class PunctuatedTranscript:
    
    def __init__(self, df_gen, df_punct, **kwargs):
        """
        kwargs:
            Just for compatibility with UnpunctuatedTranscript, which needs
            these to specify args like 'rstrip' when loading a prompt.
        """
        self.df_gen = df_gen
        self.df_punct = df_punct
        
    @property
    def df(self):
        return self.df_punct
    
    def punctuated_chunk(self, start_idx, end_idx, **kwargs):
        return self.df_punct.loc[start_idx:end_idx, :]

In [197]:
@auto_repr
class Transcript:
    
    def __init__(self, url, **kwargs):
        self.url = url
        self.id = video_id(url)
        self._transcript = self._fetch_transcripts(url, **kwargs)
        self.is_generated = isinstance(self._transcript,
                                       UnpunctuatedTranscript)
        self.start_time, self.end_time = self.df.start.ends(1)
            
    def _time_range(self, start, end, full_sentences=True):
        assert end > start, 'End time must be later than start time.'
        assert start >= 0 and end >= 0, 'Times must be non-negative.'

        df = self.df
        if start < self.start_time:
            start_idx = 0
        else:
            start_idx = df.loc[df.start <= start].index[-1]

        if end > df.start.iloc[-1]:
            end_idx = df.tail(1).index[0]
        else:
            end_idx = df.loc[df.start >= end].index[0]
        return df.iloc[start_idx:end_idx+1]
    
    def time_range(self, start, end, punctuate=False, full_sentences=True,
                   align_kwargs=None, **query_kwargs):
        chunk = self._time_range(start, end, full_sentences=full_sentences)
        return self._transcript.punctuated_chunk(*chunk.ends(1).index, 
                                                 punctuate=punctuate,
                                                 align_kwargs=align_kwargs,
                                                 **query_kwargs)
    
    @property
    def df(self):
        return self._transcript.df
    
    def _fetch_transcripts(self, url, **kwargs):
        """Wrapper to fetch youtube transcripts and create the appropriate
        transcript object depending on whether a manually generated (i.e.
        punctuated) transcript was retrieved.
        
        Parameters
        ----------
        url: str
        verbose: bool
        """
        df_gen, df_man, _ = self.get_transcripts(
            url, verbose=kwargs.get('verbose', True)
        )
        if df_man is None:
            return UnpunctuatedTranscript(df_gen, **kwargs)
        else:
            return PunctuatedTranscript(df_gen, df_man, **kwargs)
        
    @staticmethod
    def get_transcripts(url, verbose=True):
        """Fetch one or more transcripts for a youtube video given its URL.

        Parameters
        ----------
        url: str
            Don't include any channel-related suffix. E.G. use
            https://www.youtube.com/watch?v=OZbCRN3C_Hs, not
            https://www.youtube.com/watch?v=OZbCRN3C_Hs&ab_channel=BBC.
        verbose: bool
            Warn

        Returns
        -------
        DotDict: Contains keys 'id' (maps to video ID str), 'generated',
        and 'manual' (the latter two lap to pandas dfs or None if no 
        transcript was found). Manual transcripts are human-created.
        Generated transcripts are a bit lower quality and tend to lack
        punctuation.
        """
        langs = ['en', 'en-GB']
        id_ = video_id(url)
        res = {'generated': None, 'manual': None}
        trans_list = YouTubeTranscriptApi.list_transcripts(id_)
        res['generated'] = trans_list.find_generated_transcript(langs)
        try:
            res['manual'] = trans_list.find_manually_created_transcript(langs)
        except NoTranscriptFound:
            if verbose: warnings.warn('No manual transcript found.')
        if verbose:
            non_eng = [k for k, v in res.items()
                       if v and ('United Kingdom' in v.language)]
            if non_eng:
                warnings.warn(
                    f'{non_eng} {"has" if len(non_eng) == 1 else "have"} '
                    'language en-GB, not en.'
                )
        return Args(**{k: pd.DataFrame(v.fetch()) if v else v 
                       for k, v in res.items()},
                    id=id_)
    
    def __str__(self):
        return f'{type(self).__name__}(url={self.url}, '\
               f'is_generated={self.is_generated})'

In [189]:
gen_url = 'https://www.youtube.com/watch?v=AtTsn1Ia4JY&ab_channel=LukeThomas'
man_url = 'https://www.youtube.com/watch?v=NNnIGh9g6fA'

In [190]:
trans_gen = Transcript(gen_url)
trans_gen

/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:87: UserWarning: No manual transcript found.


You should probably adjust max_tokens based on the length of the input. Bumping up to engine 2 or 3 might help a little, but engine 1 is serviceable (probably best to avoid 0 though). You should probably try training a huggingface model to add punctuation instead of using gpt3 credits though.
-------------------------------------------------------------------------------



Transcript(url='https://www.youtube.com/watch?v=AtTsn1Ia4JY&ab_channel=LukeThomas')

In [191]:
trans_man = Transcript(man_url)
trans_man

Transcript(url='https://www.youtube.com/watch?v=NNnIGh9g6fA')

In [192]:
trans_man.time_range(7, 16)

,text,start,duration
1,Stanford University.,4.94,0.92
2,"This is BIO 150, isn't it?",9.36,3.37
3,"OK, just wanted to make sure.",12.73,2.74
4,So we start off with a scenario.,15.47,3.42
5,"40-year-old guy--\nquiet, suburban life.",18.89,4.06


In [193]:
trans_gen._time_range(60, 70)

,text,start,duration
35,stuff but there is,58.879,2.801
36,a little bit more to the story what was,60.160,3.440
37,dan hooker trying what was his,61.680,3.839
38,game plan here and more to the point,63.600,4.160
39,there are some finer details to that,65.519,3.121
40,finish that,67.760,3.520
41,really deserve to get mentioned and i,68.640,4.080
42,don't know exactly how far,71.280,2.640


In [194]:
trans_gen.time_range(60, 70)

,text,start,duration
35,stuff but there is,58.879,2.801
36,a little bit more to the story what was,60.160,3.440
37,dan hooker trying what was his,61.680,3.839
38,game plan here and more to the point,63.600,4.160
39,there are some finer details to that,65.519,3.121
40,finish that,67.760,3.520
41,really deserve to get mentioned and i,68.640,4.080
42,don't know exactly how far,71.280,2.640


In [177]:
res = trans_gen.time_range(60, 70, True)

[35, 36, 37, 38, 39, 40, 41, 42]


/Users/hmamin/jabberwocky/lib/jabberwocky/youtube.py:164: UserWarning: Max score < 80. Your rows may have gotten misaligned at row 37: dan hooker trying what was his
  'Max score < 80. Your rows may have gotten misaligned '


In [178]:
res

,text,start,duration
35,stuff but there is,58.879,2.801
36,a little bit more to the story. What was,60.160,3.440
37,Dan Hooker,61.680,3.839
38,"trying to do here? And more to the point,",63.600,4.160
39,there are some finer details to that,65.519,3.121
40,finish that,67.760,3.520
41,really deserve to get mentioned. And I,68.640,4.080
42,don't know exactly how far,71.280,2.640


In [179]:
res = trans_gen.time_range(50, 80, True)

[28, 29, 30, 31, 32, 33, 34]


/Users/hmamin/jabberwocky/lib/jabberwocky/youtube.py:164: UserWarning: Max score < 80. Your rows may have gotten misaligned at row 30: at like what happened to be like oh
  'Max score < 80. Your rows may have gotten misaligned '
/Users/hmamin/jabberwocky/lib/jabberwocky/youtube.py:164: UserWarning: Max score < 80. Your rows may have gotten misaligned at row 31: michael chandler him with a big ass
  'Max score < 80. Your rows may have gotten misaligned '
/Users/hmamin/jabberwocky/lib/jabberwocky/youtube.py:164: UserWarning: Max score < 80. Your rows may have gotten misaligned at row 32: punch
  'Max score < 80. Your rows may have gotten misaligned '


[43, 44, 45, 46, 47]


/Users/hmamin/jabberwocky/lib/jabberwocky/youtube.py:164: UserWarning: Max score < 80. Your rows may have gotten misaligned at row 45: he was early in bellator in his you know
  'Max score < 80. Your rows may have gotten misaligned '


In [180]:
res

,text,start,duration
28,"and um as always, the devil is in the",49.280,4.240
29,details it doesn't take a genius to look,51.840,2.960
30,at like what,53.520,2.480
31,"happened to be like, Michael Chandler, him with a",54.800,1.680
32,"big ass punch,",56.000,1.760
33,"you know, and kind of faded low and then",56.480,2.399
34,went high and then all that kind of,57.760,2.400
35,stuff but there is,58.879,2.801
36,a little bit more to the story. What was,60.160,3.440
37,Dan Hooker,61.680,3.839


In [181]:
trans_gen._transcript.df_punct.loc[25:50]

,text,start,duration
25,NaN,44.719,3.520
26,NaN,46.800,2.480
27,NaN,48.239,3.601
28,"and um as always, the devil is in the",49.280,4.240
29,details it doesn't take a genius to look,51.840,2.960
30,at like what,53.520,2.480
31,"happened to be like, Michael Chandler, him with a",54.800,1.680
32,"big ass punch,",56.000,1.760
33,"you know, and kind of faded low and then",56.480,2.399
34,went high and then all that kind of,57.760,2.400


In [184]:
# Much faster this time because it's been pre-punctuated.
res = trans_gen.time_range(50, 80, True)

In [185]:
res

,text,start,duration
28,"and um as always, the devil is in the",49.280,4.240
29,details it doesn't take a genius to look,51.840,2.960
30,at like what,53.520,2.480
31,"happened to be like, Michael Chandler, him with a",54.800,1.680
32,"big ass punch,",56.000,1.760
33,"you know, and kind of faded low and then",56.480,2.399
34,went high and then all that kind of,57.760,2.400
35,stuff but there is,58.879,2.801
36,a little bit more to the story. What was,60.160,3.440
37,Dan Hooker,61.680,3.839


In [170]:
def na_index_chunks(chunk):
    nans = chunk[chunk.text.isnull()]
    if nans.empty: return []
    last_idx = nans.index[-1]
    res = []
    curr_chunk = []
    prev = None
    for idx in nans.index:
        if prev is None or (idx == prev + 1): 
            curr_chunk.append(idx)
        else:
            res.append(curr_chunk)
            curr_chunk = [idx]
        if idx == last_idx:
            res.append(curr_chunk)
        prev = idx
    return res

In [117]:
na_index_chunks(tmp.head(3))

[[32, 33, 34]]

In [116]:
eprint([tmp.loc[row] for row in na_index_chunks(tmp)])

 0:    text  start  duration
32  NaN  56.00     1.760
33  NaN  56.48     2.399
34  NaN  57.76     2.400
 1:    text  start  duration
46  NaN  78.88     4.720
47  NaN  80.72     4.079
48  NaN  83.60     1.920


In [249]:
res = trans_gen.time_range(10, 30)

/Users/hmamin/jabberwocky/lib/jabberwocky/youtube.py:164: UserWarning: Max score < 80. Your rows may have gotten misaligned at row 11: story
  'Max score < 80. Your rows may have gotten misaligned '


[PosixPath('data/prompts/simplify_ml'),
 PosixPath('data/prompts/shortest'),
 PosixPath('data/prompts/how_to'),
 PosixPath('data/prompts/README.md'),
 PosixPath('data/prompts/punctuate'),
 PosixPath('data/prompts/tldr'),
 PosixPath('data/prompts/eli'),
 PosixPath('data/prompts/short_dates')]

In [318]:
from functools import partialmethod
from glob import glob

In [ ]:
from glob import glob

In [1541]:
class PromptManager:
    """Simple class that stores all the prompt templates and default kwargs
    so we don't need to load them repeatedly. Use this as an interface for
    performing tasks on a video Transcript object.
    """
    
    def __init__(self, *prompts):
        self.prompts = self._load_templates(set(prompts))
    
    def _load_templates(self, prompts):
        name2kwargs = {}
        dir_ = Path('data/prompts')
        paths = (dir_/p for p in prompts) if prompts else dir_.iterdir()
        for path in paths:
            if not path.is_dir(): 
                if prompts: warnings.warn(f'{path} is not a directory.')
                continue
            name2kwargs[path.stem] = load_prompt(path.stem)
        return name2kwargs
    
    def query(self, task, text, debug=False, extra_kwargs=None, **kwargs):
        """
        """
        kwargs = self.kwargs(task=task, fully_resolved=False, 
                             return_prompt=True, extra_kwargs=extra_kwargs,
                             **kwargs)
        prompt = kwargs.pop('prompt').format(text)
        if debug:
            print('prompt:\n' + prompt)
            print(spacer())
            print('kwargs:\n', kwargs)
            print(spacer())
            print('fully resolved kwargs:\n',
                  dict(bound_args(query_gpt3, [], kwargs)))
            return
        return query_gpt3(prompt, **kwargs)
    
    def kwargs(self, task, fully_resolved=True, return_prompt=False,
               extra_kwargs=None, **kwargs):
        kwargs = {**self.prompts[task], **kwargs}
        for k, v in (extra_kwargs or {}).items():
            v_cls = type(v)
            # Make a new object instead of just using get() or setdefault 
            # since the latter two methods both mutate our default kwargs.
            curr_val = v_cls(kwargs.get(k, v_cls()))
            if isinstance(v, Iterable):
                curr_val.extend(v)
            elif isinstance(v, Mapping):
                curr_val.update(v)
            else:
                raise TypeError(f'Key {k} has unrecognized type {v_cls} in '
                                '`extra_kwargs`.')
            kwargs[k] = curr_val
                
        if fully_resolved: kwargs = dict(bound_args(query_gpt3, [], kwargs))
        return kwargs if return_prompt else select(kwargs, drop=['prompt'])
    
    def prompt(self, task, text='', print_=False):
        template = self.prompts[task]['prompt']
        res = template.format(text) if text else template
        if print_:
            print(res)
        else:
            return res
    
    def __repr__(self):
        return f'{type(self).__name__}({", ".join(map(repr, self.prompts))})'
    
    def __iter__(self):
        return iter(self.prompts)

In [1524]:
manager = PromptManager('eli', 'short_dates', 'punctuate', 'eli5')
manager

short_dates: This prompt takes no input.
-------------------------------------------------------------------------------

eli: This uses the expensive davinci model and doesn't work so well without it.
-------------------------------------------------------------------------------

punctuate: You should probably adjust max_tokens based on the length of the input. Bumping up to engine 2 or 3 might help a little, but engine 1 is serviceable (probably best to avoid 0 though). You should probably try training a huggingface model to add punctuation instead of using gpt3 credits though.
-------------------------------------------------------------------------------



/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: data/prompts/eli5 is not a directory.
  app.launch_new_instance()


PromptManager('short_dates', 'eli', 'punctuate')

In [1525]:
list(manager)

['short_dates', 'eli', 'punctuate']

In [1542]:
manager = PromptManager()
manager

simplify_ml: This uses the expensive davinci model and doesn't work so well without it. Temperature is set to 0.3 but this hasn't been extensively tuned.
-------------------------------------------------------------------------------

shortest: This prompt takes no input.
-------------------------------------------------------------------------------

how_to: Should be a single line starting with the words "How to" and ending in a colon. You may need a stronger engine for good results.
-------------------------------------------------------------------------------

punctuate: You should probably adjust max_tokens based on the length of the input. Bumping up to engine 2 or 3 might help a little, but engine 1 is serviceable (probably best to avoid 0 though). You should probably try training a huggingface model to add punctuation instead of using gpt3 credits though.
-------------------------------------------------------------------------------

tldr: This sets max tokens to 64. You may 

PromptManager('simplify_ml', 'shortest', 'how_to', 'punctuate', 'tldr', 'eli', 'short_dates')

In [1527]:
manager.prompt('short_dates', print_=True)

Input: 3/1/20
Output: March 1, 2020

Input: 09-04-99
Output: September 4, 1999

Input: 11/01/2017
Output: November 1, 2017

Input: 04/11/21
Output:


In [1528]:
manager.prompt('tldr', 'abcd')

'abcd\n\ntl;dr:'

In [1529]:
manager.kwargs('tldr')

{'engine_i': 2,
 'logprobs': None,
 'max_tokens': 64,
 'mock': False,
 'return_full': False,
 'stream': False,
 'strip_output': True,
 'temperature': 0.3}

In [1530]:
manager.kwargs('tldr', fully_resolved=False)

{'engine_i': 2, 'max_tokens': 64, 'temperature': 0.3}

In [1531]:
manager.kwargs('tldr', fully_resolved=False, return_prompt=True)

{'engine_i': 2, 'max_tokens': 64, 'prompt': '{}\n\ntl;dr:', 'temperature': 0.3}

In [1532]:
manager.kwargs('tldr', fully_resolved=True, return_prompt=True)

{'engine_i': 2,
 'logprobs': None,
 'max_tokens': 64,
 'mock': False,
 'prompt': '{}\n\ntl;dr:',
 'return_full': False,
 'stream': False,
 'strip_output': True,
 'temperature': 0.3}

In [1533]:
manager.kwargs('tldr', extra_kwargs={'stop': ['stop1', 'stop2']})

{'engine_i': 2,
 'logprobs': None,
 'max_tokens': 64,
 'mock': False,
 'return_full': False,
 'stop': ['stop1', 'stop2'],
 'stream': False,
 'strip_output': True,
 'temperature': 0.3}

In [1536]:
manager.kwargs('tldr', extra_kwargs={'stop': ['stop1', 'stop2']}, 
               logit_bias={100: -5})

{'engine_i': 2,
 'logit_bias': {100: -5},
 'logprobs': None,
 'max_tokens': 64,
 'mock': False,
 'return_full': False,
 'stop': ['stop1', 'stop2'],
 'stream': False,
 'strip_output': True,
 'temperature': 0.3}

In [1537]:
manager.kwargs('tldr')

{'engine_i': 2,
 'logprobs': None,
 'max_tokens': 64,
 'mock': False,
 'return_full': False,
 'stream': False,
 'strip_output': True,
 'temperature': 0.3}

In [1538]:
txt = """Attention enables the network to learn dynamic linear layers,
essentially encoding the input sequence as a directed, weighted graph
which we represent using a learned adjacency matrix.""".replace('\n', ' ')
manager.query('eli', txt, debug=True, stream=True, engine_i=1)

prompt:
Attention enables the network to learn dynamic linear layers, essentially encoding the input sequence as a directed, weighted graph which we represent using a learned adjacency matrix. 

I rephrased this for my daughter, in plain language a second grader can understand:

-------------------------------------------------------------------------------

kwargs:
 {'engine_i': 1, 'temperature': 0.3, 'stop': ['I rephrased this for my daughter, in plain language', 'prompt:'], 'stream': True}

-------------------------------------------------------------------------------

fully resolved kwargs:
 {'engine_i': 1, 'temperature': 0.3, 'max_tokens': 50, 'logprobs': None, 'stream': True, 'mock': False, 'return_full': False, 'strip_output': True, 'stop': ['I rephrased this for my daughter, in plain language', 'prompt:']}


In [1539]:
txt = """Attention enables the network to learn dynamic linear layers,
essentially encoding the input sequence as a directed, weighted graph
which we represent using a learned adjacency matrix.""".replace('\n', ' ')
manager.query('eli', txt, debug=True, stream=True, engine_i=1, 
              extra_kwargs={'stop': ['\n\n']})

prompt:
Attention enables the network to learn dynamic linear layers, essentially encoding the input sequence as a directed, weighted graph which we represent using a learned adjacency matrix. 

I rephrased this for my daughter, in plain language a second grader can understand:

-------------------------------------------------------------------------------

kwargs:
 {'engine_i': 1, 'temperature': 0.3, 'stop': ['I rephrased this for my daughter, in plain language', 'prompt:', '\n\n'], 'stream': True}

-------------------------------------------------------------------------------

fully resolved kwargs:
 {'engine_i': 1, 'temperature': 0.3, 'max_tokens': 50, 'logprobs': None, 'stream': True, 'mock': False, 'return_full': False, 'strip_output': True, 'stop': ['I rephrased this for my daughter, in plain language', 'prompt:', '\n\n']}


In [1540]:
manager.prompts['eli']

{'engine_i': 3,
 'prompt': '{} \n\nI rephrased this for my daughter, in plain language a second grader can understand:',
 'stop': ['I rephrased this for my daughter, in plain language', 'prompt:'],
 'temperature': 0.3}

In [1548]:
txt = """Attention enables the network to learn dynamic linear layers,
essentially encoding the input sequence as a directed, weighted graph
which we represent using a learned adjacency matrix.""".replace('\n', ' ')
res = manager.query('eli', txt, max_tokens=100, temperature=.2)

In [1549]:
res

('Attention enables the network to learn dynamic linear layers, essentially encoding the input sequence as a directed, weighted graph which we represent using a learned adjacency matrix. \n\nI rephrased this for my daughter, in plain language a second grader can understand:',
 '"Attention is a way for the network to pay attention to what it thinks is important in the input sequence. It does this by looking at the input sequence and deciding what parts of it are important. It then uses this information to decide what parts of the output sequence should be important."\n\nThe attention mechanism is a way for the network to decide what parts of the input sequence are important. It does this by looking at the input sequence and deciding what parts of it are important.')

In [1384]:
txt = """Attention enables the network to learn dynamic linear layers,
essentially encoding the input sequence as a directed, weighted graph
which we represent using a learned adjacency matrix.""".replace('\n', ' ')
for chunk in manager.query('eli', txt, max_tokens=200, stream=True):
    print(chunk)



The
network
learns
to
recognize
patterns
in
the
input
sequence
.
It
learns
to
recognize
patterns
by
looking
at
the
input
sequence
and
seeing
how
it
is
similar
to
other
input
sequences
.


The
network
learns
to
recognize
patterns
in
the
input
sequence
.
It
learns
to
recognize
patterns
by
looking
at
the
input
sequence
and
seeing
how
it
is
similar
to
other
input
sequences
.


The
network
learns
to
recognize
patterns
in
the
input
sequence
.
It
learns
to
recognize
patterns
by
looking
at
the
input
sequence
and
seeing
how
it
is
similar
to
other
input
sequences
.


The
network
learns
to
recognize
patterns
in
the
input
sequence
.
It
learns
to
recognize
patterns
by
looking
at
the
input
sequence
and
seeing
how
it
is
similar
to
other
input
sequences
.


The
network
learns
to
recognize
patterns
in
the
input
sequence
.
It
learns
to
recognize
patterns
by
looking
at
the
input
sequence
and
seeing
how
it
is
similar
to
other
input
sequences
.


The
network
learns
to
recognize
patterns
in
the
input
sequ

In [1416]:
manager.kwargs('eli')

{'engine_i': 3,
 'stop': 'I rephrased this for my daughter, in plain language',
 'temperature': 0.3}

In [1418]:
manager.kwargs('punctuate')

{'engine_i': 1,
 'stop': ['Passage: ', 'Passage with punctuation: '],
 'strip_output': True,
 'temperature': 0.1}

In [1420]:
txt = """Attention enables the network to learn dynamic linear layers,
essentially encoding the input sequence as a directed, weighted graph
which we represent using a learned adjacency matrix.""".replace('\n', ' ')
# _, res = 
manager.query('eli', txt, max_tokens=100, debug=True)

eli Attention enables the network to learn dynamic linear layers, essentially encoding the input sequence as a directed, weighted graph which we represent using a learned adjacency matrix.
prompt:
Attention enables the network to learn dynamic linear layers, essentially encoding the input sequence as a directed, weighted graph which we represent using a learned adjacency matrix. 

I rephrased this for my daughter, in plain language a second grader can understand:
kwargs:
 {'engine_i': 3, 'temperature': 0.3, 'stop': 'I rephrased this for my daughter, in plain language', 'max_tokens': 100}


In [1388]:
print(res)

"The network learns to pay attention to the things that matter most, and ignore the rest."

The attention mechanism is a key ingredient to making neural networks useful for natural language tasks.

The attention mechanism is a key ingredient to making neural networks useful for natural language tasks.

In the paper, the authors used the attention mechanism to improve the state-of-the-art performance on a number of natural language tasks, including machine translation, question answering, and sentence summarization.

The attention mechanism is a key ingredient to making neural networks useful for natural language tasks.

In the paper, the authors used the attention mechanism to improve the state-of-the-art performance on a number of natural language tasks, including machine translation, question answering, and sentence summarization.

The attention mechanism is a key ingredient to making neural networks useful for natural language tasks.

In the paper, the authors used the attention mec

In [1390]:
manager

PromptManager(prompts={'simplify_ml': {'engine_i': 3, 'temperature': 0.3, 'stop': 'Passage:', 'prompt': "My fifth grader asked me what this passage means. I rephrased it for him, in plain language a fifth grader can understand.\n\nPassage:\nHere we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art finetuning approaches. Specifically, we train GPT-3, an autoregressive language model with 175 billion parameters, 10x more than any previous non-sparse language model, and test its performance in the few shot setting. For all tasks, GPT-3 is applied with any gradient updates or fine-tuning, with tasks and few-shot demonstrations specified purely via text interaction with the model.\nSummary:\nLarger language models can do well on new tasks they weren't trained on. We trained the biggest language model ever and tested it on new tasks, and it did really well. This model can perform new 

In [284]:
manager.query('eli', 'Batch normalization decouples layer outputs from one another '
             'and reduces internal covariate shift. It leads to faster '
             'optimization by enabling the neural network to undergo stable '
             'training even with large learning rates.', engine_i=3)

('Batch normalization decouples layer outputs from one another and reduces internal covariate shift. It leads to faster optimization by enabling the neural network to undergo stable training even with large learning rates. \n\nI rephrased this for my daughter, in plain language a second grader can understand:',
 'Batch normalization is a way to make the learning process faster.\n\nBatch normalization is a technique that allows you to use larger learning rates.\n\nBatch normalization makes the neural network more stable.\n\nB')

In [343]:
params(pfoo)

{'a': <Parameter "a">,
 'args': <Parameter "*args">,
 'b': <Parameter "b=99">,
 'kwargs': <Parameter "**kwargs">}

In [903]:
def foo(a, a2, b=2, *args, c=3, **kwargs):
    print('a', a)
    print('a2', a2)
    print('b', b)
    print('args', args)
    print('c', c)
    print('kwargs', kwargs)

In [449]:
print(signature(foo))
print(foo.__name__)
print('def', foo.__defaults__)
print('kw def', foo.__kwdefaults__)

(a, b=2, *args, c=3, **kwargs)
foo
def (2,)
kw def {'c': 3}


In [450]:
pfoo = partial(foo, b=99, d='d')
print(signature(pfoo))
print(pfoo.__name__)

(a, *, b=99, c=3, **kwargs)


AttributeError: 'functools.partial' object has no attribute '__name__'

In [451]:
dfoo = partial(foo, b=99, d='d')
dfoo = update_wrapper(dfoo, foo)
dfoo.__dir__ = foo.__dir__
print(signature(dfoo))
print(dfoo.__name__)

(a, b=2, *args, c=3, **kwargs)
foo


In [398]:
params(pfoo)

{'a': <Parameter "a">,
 'args': <Parameter "*args">,
 'b': <Parameter "b=99">,
 'kwargs': <Parameter "**kwargs">}

In [358]:
dfoo()

TypeError: foo() missing 1 required positional argument: 'a'

In [460]:
from types import FunctionType, MethodType

In [461]:
{**(foo.__defaults__ or {})}

TypeError: 'tuple' object is not a mapping

In [462]:
foo.__defaults__

(2,)

In [463]:
bar.__kwdefaults__

In [476]:
dir(pfoo)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'args',
 'func',
 'keywords']

In [477]:
pfoo.keywords

{'b': 99, 'd': 'd'}

In [473]:
pfoo.func.__defaults__

(2,)

In [474]:
pfoo.func.__kwdefaults__

{'c': 3}

In [475]:
pfoo.args

()

# TODO

- if we pass in a value for a param w/ no default but not for another, param order can get messed up (should have all params w/out defaults first)  
    IDEAS:  
    - to be safe, could put all args after \*args (if present), i.e. make them keyword only.
    - after making new_pars, rm all w/ empty default and prepend
    - sort new_pars by (kind, default) while replacing inspect._empty with float('-inf'). Minor problem: this might fail if another param has default float('-inf') too.
- when passing in kwargs, how do we use them in func? Because we added them to sig and kwargs have no defaults, so func would have to guess what args are available before creating partial  
    IDEAS:  
    - update signature but don't actually add those param defaults to the func. I think this would suit this particular use case (we can (and do) get the appropriate kwargs inside the base method) but it wouldn't be good for a generalized partial method. Maybe these are two separate goals. 

In [744]:
from inspect import _empty

In [1088]:
def attach_repr(func, repr_func=None, str_func=None, name=None):
    class CustomReprFunction:
        def __init__(self, func, repr_func, str_func):
            self.func = func
            self.repr_func = repr_func
            self.str_func = str_func
            update_wrapper(self, func)
            self.__name__ = ifnone(name, func.__name__)
        
        def __repr__(self):
            if self.repr_func:
                return self.repr_func(self)
            return repr(self.func)
        
        def __call__(self, *args, **kwargs):
            return self.func(*args, **kwargs)
            
        def __str__(self):
            if self.str_func:
                return self.str_func(self)
            return repr(self.func)
        
    func_ = CustomReprFunction(func, repr_func, str_func)
    return func_

In [1134]:
tmp = attach_repr(foo, name='bar')
tmp, str(tmp), tmp.__name__

(<function foo at 0x123fbf730>, '<function foo at 0x123fbf730>', 'bar')

In [1010]:
def getindex(arr, val, default=-1):
    return arr.index(val) if val in arr else default

In [1114]:
def rigorous_partial(func, name=None, **kwargs):
    func = copy_func(func)
    old_sig = signature(func)
    # Track names of positional args in old function since this affects the 
    # order args must be passed in if var_positional parameters (*args) are
    # present.
    old_pos_pars = []
    new_pars = []
    kwargs_name = ''
    args_name = ''
    for k, v in old_sig.parameters.items():
        if v.kind == 0:
            raise NotImplementedError(
                'rigorous_partial does not support functions with positional '
                'only parameters.'
            )
        elif v.kind == 2:
            args_name = k
        elif v.kind == 4:
            kwargs_name = k
            break
            
        if v.kind <= 2:
            old_pos_pars.append(k)

        # Assign default value from newly specified kwargs if provided.
        if k in kwargs:
            default = kwargs.pop(k)
            kind = 3
        else:
            default = v.default
            kind = v.kind
        param = inspect.Parameter(k, kind, default=default)
        new_pars.append(param)
        
    # Remaining kwargs only: those that were not present in func's signature.
    # Require that they be keyword only since ordering can cause issues
    # (updating signature affects what we see but doesn't seem to affect the 
    # actual order args are passed in, presumably due to old __code__ object).
    for k, v in kwargs.items():
        param = inspect.Parameter(k, 3, default=v)
        new_pars.append(param)
    if kwargs_name:
        new_pars.append(inspect.Parameter(kwargs_name, 4))
        
    # Ensure we don't accidentally place any parameters with defaults ahead
    # of those without them. Third item in tuple is a tiebreaker (defaults to
    # original function's parameter order).
    old_names = [p for p in old_sig.parameters]
    new_pars.sort(key=lambda x: (x.kind, x.default != _empty, 
                                 getindex(old_names, x.name, float('inf'))))

    # TODO: might need to attach these to the wrapped func instead? Can't 
    # access these outside this function. But we use it inside wrapper so that
    # could cause other problems.
    func.__defaults__ = tuple(p.default for p in new_pars if p.kind < 3 
                              and p.default != _empty)
    func.__kwdefaults__ = {p.name: p.default for p in new_pars if p.kind == 3}
    func.__signature__ = old_sig.replace(parameters=new_pars)
    if name: func.__name__ = name
    
    @wraps(func)
    def wrapper(*args, **new_kwargs):
        final_kwargs = bound_args(func, args, 
                                  {**func.__kwdefaults__, **new_kwargs})
        final_args = {name: final_kwargs.pop(name) for name in old_pos_pars}
        final_star_args = final_args.pop(args_name, [])
        final_kwargs = select(final_kwargs, drop=list(final_args))
        print('old_pos_pars', old_pos_pars)
        print('final_kwargs', final_kwargs)
        print('final_args', final_args)
        print('final star args', final_star_args)
        return func(*final_args.values(), *final_star_args, **final_kwargs)
    wrapper.__kwdefaults__ = func.__kwdefaults__
    wrapper.__defaults__ = func.__defaults__
    return wrapper

In [1352]:
class Partial:
    
    def __init__(self, func, name=None, **kwargs):
        self.func = copy_func(func)
        self.old_name = func.__name__

        # Track names of positional args in old function since this affects 
        # the order args must be passed in if var_positional parameters 
        # (*args) are present.
        self.old_pos_pars = []
        self.kwargs_name = ''
        self.args_name = ''
        new_pars = []
        old_sig = signature(self.func)
        for k, v in old_sig.parameters.items():
            # Check parameter kind for error handling and argument resolution
            # in __call__.
            if v.kind == 0:
                raise NotImplementedError(
                    'rigorous_partial does not support functions with '
                    'positional only parameters.'
                )
            elif v.kind == 2:
                self.args_name = k
            elif v.kind == 4:
                self.kwargs_name = k
                break

            if v.kind <= 2:
                self.old_pos_pars.append(k)

            # Assign default value from newly specified kwargs if provided.
            if k in kwargs:
                default = kwargs.pop(k)
                kind = 3
            else:
                default = v.default
                kind = v.kind
            param = inspect.Parameter(k, kind, default=default)
            new_pars.append(param)

        # Remaining kwargs only: those that were not present in func's 
        # signature. Require that they be keyword only since ordering can
        # cause issues (updating signature affects what we see but doesn't 
        # seem to affect the actual order args are passed in, presumably due 
        # to old __code__ object).
        for k, v in kwargs.items():
            param = inspect.Parameter(k, 3, default=v)
            new_pars.append(param)
        if self.kwargs_name:
            new_pars.append(inspect.Parameter(self.kwargs_name, 4))

        # Ensure we don't accidentally place any parameters with defaults 
        # ahead of those without them. Third item in tuple is a tiebreaker 
        # (defaults to original function's parameter order).
        old_names = [p for p in old_sig.parameters]
        new_pars.sort(
            key=lambda x: (x.kind, x.default != _empty, 
                           getindex(old_names, x.name, float('inf')))
        )

        # I honestly forget why we need to set the attribute on self.func too,
        # I just remember it was needed to resolve a bug (I think it was 
        # related to *args resolution).
        self.__signature__ = self.func.__signature__ = old_sig.replace(
            parameters=new_pars
        )
        self.__defaults__ = tuple(p.default for p in new_pars if p.kind < 3 
                                  and p.default != _empty)
        self.__kwdefaults__ = {p.name: p.default for p in new_pars
                               if p.kind == 3}
        if name: self.func.__name__ = name
        update_wrapper(self, self.func)
    
    def __call__(self, *args, **new_kwargs):
        tmp_kwargs = bound_args(self.func, args, 
                                {**self.__kwdefaults__, **new_kwargs})
        final_args = {name: tmp_kwargs.pop(name) 
                      for name in self.old_pos_pars}
        final_star_args = final_args.pop(self.args_name, [])
        final_kwargs = select(tmp_kwargs, drop=list(final_args))
        return self.func(*final_args.values(), *final_star_args,
                         **final_kwargs)

    def __repr__(self):
        return repr(self.func).replace(self.old_name, self.__name__)
    
    def __str__(self):
        return str(self.func).replace(self.old_name, self.__name__)

In [1358]:
partial(foo, a='a')

functools.partial(<function foo at 0x123fbf730>, a='a')

In [1359]:
Partial(foo, a='a')

<function foo at 0x123ed1400>

In [ ]:
foo()

In [1208]:
new_v2 = Partial(foo, a=False, x=9, y=8, z='zzzz')
new_v2

<function foo at 0x123e418c8>

In [1209]:
new_v2('a2', 'b_val', 'some', 'args', 'more', new_kwarg='q')

a False
a2 a2
b b_val
args ('some', 'args', 'more')
c 3
kwargs {'x': 9, 'y': 8, 'z': 'zzzz', 'new_kwarg': 'q'}


In [1210]:
new_v2 = Partial(foo, name='a_new_name', a=False, x=9, y=8, z='zzzz')
new_v2

<function a_new_name at 0x123e50378>

In [1211]:
new_v2('a2', 'b_val', 'some', 'args', 'more', new_kwarg='q')

a False
a2 a2
b b_val
args ('some', 'args', 'more')
c 3
kwargs {'x': 9, 'y': 8, 'z': 'zzzz', 'new_kwarg': 'q'}


In [1212]:
new_v2.__name__, new_v2.__defaults__, new_v2.__kwdefaults__

('a_new_name', (2,), {'a': False, 'c': 3, 'x': 9, 'y': 8, 'z': 'zzzz'})

In [1213]:
str(new_v2)

'<function a_new_name at 0x123e50378>'

In [1214]:
inspect.isfunction(new_v2)

False

In [1215]:
inspect.isfunction(foo)

True

In [1217]:
print(inspect.getsource(foo))

def foo(a, a2, b=2, *args, c=3, **kwargs):
    print('a', a)
    print('a2', a2)
    print('b', b)
    print('args', args)
    print('c', c)
    print('kwargs', kwargs)



In [1218]:
# Tbh not sure what I'd want this to return. Ideally I guess we'd have
# rewritten the actual definition but I don't think this is possible without
# rebuilding the __code__ object, which I remember required using a largely
# undocumented function whose interface (purportedly) frequently changes.
print(inspect.getsource(new_v2))

def foo(a, a2, b=2, *args, c=3, **kwargs):
    print('a', a)
    print('a2', a2)
    print('b', b)
    print('args', args)
    print('c', c)
    print('kwargs', kwargs)



In [1203]:
eprint(params(new_v2).items())

 0: ('a2', <Parameter "a2">)
 1: ('b', <Parameter "b=2">)
 2: ('args', <Parameter "*args">)
 3: ('a', <Parameter "a=False">)
 4: ('c', <Parameter "c=3">)
 5: ('x', <Parameter "x=9">)
 6: ('y', <Parameter "y=8">)
 7: ('z', <Parameter "z='zzzz'">)
 8: ('kwargs', <Parameter "**kwargs">)


In [1200]:
signature(new_v2)

<Signature (a2, b=2, *args, a=False, c=3, x=9, y=8, z='zzzz', **kwargs)>

In [1102]:
signature(foo)

<Signature (a, a2, b=2, *args, c=3, **kwargs)>

In [1103]:
print(inspect.getsource(foo))

def foo(a, a2, b=2, *args, c=3, **kwargs):
    print('a', a)
    print('a2', a2)
    print('b', b)
    print('args', args)
    print('c', c)
    print('kwargs', kwargs)



In [1104]:
eprint({k: v.kind for k, v in params(foo).items()}.items())

 0: ('a', <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>)
 1: ('a2', <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>)
 2: ('b', <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>)
 3: ('args', <_ParameterKind.VAR_POSITIONAL: 2>)
 4: ('c', <_ParameterKind.KEYWORD_ONLY: 3>)
 5: ('kwargs', <_ParameterKind.VAR_KEYWORD: 4>)


In [1105]:
signature(foo)

<Signature (a, a2, b=2, *args, c=3, **kwargs)>

In [1106]:
foo.__defaults__

(2,)

In [1107]:
foo.__kwdefaults__

{'c': 3}

In [1108]:
# Should have a=aaaa, a2=a2, b=2, args=(), c=3, kwargs={}.
foo('aaaa', 'a2')

a aaaa
a2 a2
b 2
args ()
c 3
kwargs {}


In [1236]:
new = Partial(foo, a2=111, z=99)
new

<function foo at 0x123e8ac80>

In [1237]:
# Should look like Signature(a, b=2, *args, a2=111, c=3, z=99, **kwargs).
signature(new)

<Signature (a, b=2, *args, a2=111, c=3, z=99, **kwargs)>

In [1238]:
eprint({k: (v.kind, v.default) for k, v in params(new).items()}.items())

 0: ('a', (<_ParameterKind.POSITIONAL_OR_KEYWORD: 1>, <class 'inspect._empty'>))
 1: ('b', (<_ParameterKind.POSITIONAL_OR_KEYWORD: 1>, 2))
 2: ('args', (<_ParameterKind.VAR_POSITIONAL: 2>, <class 'inspect._empty'>))
 3: ('a2', (<_ParameterKind.KEYWORD_ONLY: 3>, 111))
 4: ('c', (<_ParameterKind.KEYWORD_ONLY: 3>, 3))
 5: ('z', (<_ParameterKind.KEYWORD_ONLY: 3>, 99))
 6: ('kwargs', (<_ParameterKind.VAR_KEYWORD: 4>, <class 'inspect._empty'>))


In [1239]:
new.__defaults__

(2,)

In [1240]:
new.__kwdefaults__

{'a2': 111, 'c': 3, 'z': 99}

In [1241]:
# Should be b=2, a2=111
new('a')

a a
a2 111
b 2
args ()
c 3
kwargs {'z': 99}


In [1242]:
# Should be b='b', a2=111
new('a', 'b')

a a
a2 111
b b
args ()
c 3
kwargs {'z': 99}


In [1243]:
# Should be b=2, a2='a2'
new('a', a2='a2')

a a
a2 a2
b 2
args ()
c 3
kwargs {'z': 99}


In [1244]:
new(1, c='ccc', d='d_kwarg')

a 1
a2 111
b 2
args ()
c ccc
kwargs {'z': 99, 'd': 'd_kwarg'}


In [1248]:
new(1, 'b', 0, 1, 2, 3, 4, 5, c='ccc', d='d_kwarg')

a 1
a2 111
b b
args (0, 1, 2, 3, 4, 5)
c ccc
kwargs {'z': 99, 'd': 'd_kwarg'}


In [1249]:
new('a', 'a2', -1, 0, 1, 2, 3, 4, 5)

a a
a2 111
b a2
args (-1, 0, 1, 2, 3, 4, 5)
c 3
kwargs {'z': 99}


In [1250]:
new('a', 'a2', -1, 0, 1, 2, 3, 4, 5, z='nnn')

a a
a2 111
b a2
args (-1, 0, 1, 2, 3, 4, 5)
c 3
kwargs {'z': 'nnn'}


In [1251]:
new.__defaults__, new.__wrapped__.__defaults__

((2,), (2,))

In [1252]:
new.__kwdefaults__, new.__wrapped__.__kwdefaults__

({'a2': 111, 'c': 3, 'z': 99}, {'c': 3})

In [1254]:
new_named = Partial(foo, name='my_new_func', b=888, x='xxx', t='ttt')
new_named

<function my_new_func at 0x123ed11e0>

In [1255]:
new_named.__name__

'my_new_func'

In [1256]:
func_name(new_named)

'my_new_func'

In [1257]:
new_named.__wrapped__.__name__

'my_new_func'

In [1258]:
func_name(new_named.__wrapped__)

'my_new_func'

In [1353]:
# Testing example from stackoverflow where functools version acts weirdly. 
# https://stackoverflow.com/questions/38975975/python-partial-with-keyword-arguments
def baz(x=1, y=2, z=3):
    return f'x:{x}, y:{y}, z:{z}'

# Looks like my Partial works as expected here and functools.partial still
# does not.
zz = range(10)
f0 = Partial(baz, x=0, y=-6)
eprint(list(map(f0, zz)))
print(spacer())

f1 = partial(baz, x=0, y=-6)
eprint(list(map(f1, zz)))

 0: x:0, y:-6, z:0
 1: x:0, y:-6, z:1
 2: x:0, y:-6, z:2
 3: x:0, y:-6, z:3
 4: x:0, y:-6, z:4
 5: x:0, y:-6, z:5
 6: x:0, y:-6, z:6
 7: x:0, y:-6, z:7
 8: x:0, y:-6, z:8
 9: x:0, y:-6, z:9

-------------------------------------------------------------------------------



TypeError: baz() got multiple values for argument 'x'

In [1356]:
signature(f0)

<Signature (z=3, *, x=0, y=-6)>